## Funktionen

Neben den Importen werden auch drei Funktionen definiert, die über die verwendeten Notebooks hinweg benötigt werden. Für die Verwendung muss ebenfalls dieses Notebook zuerst ausgeführt werden.

> %run Util/01_functions.ipynb

Die erste Funktion, die innerhalb dieses Notebooks definiert wird, befasst sich mit der Umwandlung von einer der drei genutzten Darstellungsweisen in eine andere. Bei diesen Darstellungsweisen handelt es sich um:
* Die Forsyth-Edward-Notation (FEN). Diese besteht aus insgesamt sechs Teilstrings, die zur Beschreibung einer Schachpartie benötigt werden. Eine FEN besteht aus folgenden Elementen:
    * Die Platzierung der Spielfiguren auf dem Schachbrett.
    * Die Farbe, die gerade am Zug ist.
    * Die Möglichkeiten, noch eine Rochade durchzuführen
    * Die Felder hinter einem Bauern, der per En Passant geschlagen werden können.
    * Die Anzahl an Halbzügen, seitdem die letzte Figur geschlagen wurde oder ein Bauer vorgestoßen ist.
    * Die Anzahl an durchgeführten Zügen.
* Die gekürzte FEN-Schreibweise. Diese besteht im Gegensatz zur klassischen FEN-Schreibweise nur aus der Platzierung der Spielfiguren und der Farbe, die aktuell am Zug ist.
* Und ein Zwei-Tupel. Dieses Tupel besitzt als ersten Wert einen booleschen Wert, der die aktuelle Farbe darstellt, die am Zug ist. Der zweite Eintrag stellt eine ASCII-Repräsentation eines gesamten Schachbretts dar.

Die Funktion ``get_board_and_turn`` erstellt nun für eine mitgegebene ``fen`` die gekürzte FEN-Schreibweise und gibt diese zurück.

In [ ]:
def get_board_and_turn(fen):
    split = fen.split()
    short_fen = str(split[0]) + " " + str(split[1])
    return short_fen

Als Nächstes wird eine Funktion definiert, die die Suche innerhalb einer ``sequence`` ermöglicht. Diese ``sequence`` ist eine Liste bestehend aus Mengen. In diesen Mengen stehen die Spielsituationen in einer der drei zuvor erwähnten Darstellungsweisen. Bei der ``situation`` muss darauf geachtet werden, dass sie die gleiche Darstellung besitzt wie die Elemente in der ``sequence``. Sofern die Funktion ``find_situation_in_sequence`` die Spielsituation in einer der Mengen findet, wird der Mengenindex der Liste zurückgegeben. Andernfalls lautet der Rückgabewert -1.

In [1]:
def find_situation_in_sequence(situation, sequence):
    for i in range(len(sequence)):
        if situation in sequence[i]:
            return i
    return -1

Im weiteren Verlauf dieser Studienarbeit werden in dem Notebook ``02_calculation.ipynb`` Ergebnisse berechnet, die für eine spätere Validierung benötigt werden. Da die Ergebnisse eine beachtliche Größe (genaue Zahl) besitzen, werden diese lokal abgespeichert und müssen für eine spätere Verwendung erneut geladen werden. Hierfür wurde die Funktion ``load_s_n_fens`` definiert. Diese nimmt den Dateinamen (``filename``) einer ``.chessTest`` entgegen, die in dem Ordner ``S_n_Results`` gespeichert werden. Aus dieser Datei werden zwei Listen von Mengen generiert, die Spielsituation in sowohl der FEN-Schreibweise (``s_n_sequence_fen``) und der gekürzten FEN-Schreibweise(``s_n_sequence_fen``) beinhalten.

In [ ]:
def load_s_n_fens(filename):
    s_n_sequence_fen = []
    s_n_sequence_short_fen = []
    with (open("S_n_Results/" + filename + ".chessTest", "rb")) as test_fens:
        tmp = pickle.loads(test_fens.read())
        for item in tmp:
            tmp_set_fen = set()
            tmp_set_short = set()
            for fen in item:
                tmp_set_fen.add(fen)
                tmp_set_short.add(get_board_and_turn(fen))
            s_n_sequence_fen.append(tmp_set_fen)
            s_n_sequence_short_fen.append(tmp_set_short)
    return s_n_sequence_fen, s_n_sequence_short_fen